# Ансамбли

### OzonMasters, "Машинное обучение 1"

В этом ноутбуке вам предлагается реализовать алгоритмы бустинга и бэггинга.

In [1]:
import numpy as np
import numpy.testing as npt
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.datasets import make_classification

## 1. Сэмплирование случайных объектов и признаков

Во многих ансамблевых алгоритмах используется прием, заключающийся в обучении на случайной подвыборке объектов или на случайном подмножестве признаков.

Реализуйте класс, который будет упрощать семплирование различных подмассивов данных: `BaseSampler`.

В классе `BaseSampler` реализуйте метод `sample_indices` который по числу сущностей `n_objects` возращает случайную подвыборку индексов. Используйте атрибут `self.random_state`, чтобы результаты семпплирования воспроиводились. Используйте атрибут `self.bootstrap`, если нужно выбрать случайную подвыборку с возвращением.

У класса `ObjectSampler` реализован метод `sample`, который возвращает случайную подвыборку объектов обучения и ответы для них.

В классе `FeaturesSampler` реализован метод `sample`, который возвращает случайную подвыборку признаков для объектов.

## 2. Бэггинг (2 балла)

Суть бэггинга заключается в обучении нескольких "слабых" базовых моделей и объединении их в одну модель, обладающую бОльшей обобщающей способностью. Каждая базовая модель обучается на случайно выбранном подмножестве объектов и на случайно выбранном подмножестве признаков для этих объектов.

Ниже вам предлагается реализовать несколько методов класса `Bagger`:
* `fit` - обучение базовых моделей
* `predict_proba` - вычисление вероятностей ответов.

Тогда алгоритм случайный лес будет бэггингом над решающими деревьями. Реализация случайного веса представлена в классе `RandomForestClassifier`.

## 3. Градиентный бустинг (2 балла)

Бустинг последовательно обучает набор базовых моделей таким образом, что каждая следующая модель пытается исправить ошибки работы предыдущей модели. Логика того, как учитываются ошибки предыдущей модели может быть разной. В алгоритме градиентного бустинга каждая следующая модель обучается на "невязках" предыдущей модели, минимизируя итоговую функцию потерь. У каждого следующего алгоритма вычисляется вес $\alpha$, с которым он входит в ансамбль. Также есть параметр скорости обучения (learning rate), который не позволяет алгоритму переобучитсья. Вес $\alpha$ можно находить, используя одномерную оптимизацию. Можно записать процедуру обучения по шагам (будем рассматривать случай бинарной классификации c метками классов {0,1}, чтобы не усложнять жизнь):
1. Настройка базового алгоритма $b_0$.
    
    Алгоритм настраиваются на $y$ с помощью функции MSE.
    
    
2. Будем обозначать текущий небазовый алгоритм - $a$:
    
    $$a_i(x) = \sum_{j=0}^i \alpha_j b_j(x) $$
    
3. Настройка базового алгоритма $b_i$ (обычно это регрессионное дерево):
    
    $$b_i = \arg \min_b \sum_{j=1}^l (b(x_j) + \nabla L(a_{i-1}(x_j), y))^2,$$
    т.е. выход очередного базового алгоритма подстраивается под антиградиент функции потерь
    
4. Настройка веса базового алгоритма $\alpha_i$:
    
    $$\alpha_i = \min_{\alpha > 0} \sum_{j=1}^l L(a_{i-1} + \alpha b_i(x_j), y) $$
    
В случае классфикации будем использовать логистическую функцию потерь. Немного упростим ее:

$$L = -y\log\sigma(a) - (1-y)\log(1 - \sigma(a)) = -\log(1 - \sigma(a)) - y \log \frac{\sigma(a)}{1 - \sigma(a)},$$
где $\sigma$ - функция сигмоиды. Ответ после очередного базового алгоритма надо прогонять через сигмоиду, т.к. не гарантируется, что ответы будут лежать на [0,1] - в этом особенность базового алгоритма (который является регрессионным).

Преобразуем:
$$\log (1 - \sigma(a)) = \log \frac{1}{1 + \exp(a)} = -\log(1 + \exp(a)) $$

$$\log (\frac{\sigma(a)}{1 - \sigma(a)}) = \log(\exp(a)) = a $$
 
Таким образом:

$$L = -ya + \log(1 + \exp(a))$$

Тогда будем вычислять градиент как:
 
$$\nabla L = - y + \sigma(a)$$

В классе `Booster` реализуйте методы:
* `_fit_first_estimator` – построение первой модели (первого приближения данных);
* `fit` – обучение алгоритма градиентного бустинга (обучение первой и последующих базовых моделей);
* `predict` – получение предсказаний алгоритма градиентного бустинга.

В классе `GradientBoostingClassifier` реализуйте методы:
* `_fit_base_estimator` - обучение очередной базовой модели;
* `_gradient` - расчет градиента функции ошибки;
* `_loss` - расчет функции ошибки (для одномерно оптимизации).

## Эксперименты (3 балла)

Скачайте датасейт для экспериментов: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

Колонка с ответами - RainTommorow.

In [2]:
%load_ext autoreload
%autoreload 2

from ensemble import RandomForestClassifier, GradientBoostingClassifier

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('weatherAUS.csv')
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


Выделите признаки год/месяц/день:

In [5]:
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')

In [6]:
data['year'] = data['Date'].dt.year
data['month'] = data['Date'].dt.month
data['day'] = data['Date'].dt.day

In [7]:
data = data.drop("Date", axis = 1)

Посмотрим какие года есть в выборке:

Разделите выборку на три части (train, val и test) по временному принципу:
    
* train - 2007-2014
* val - 2015
* test - 2016-2017

In [8]:
indexes = {
    'train': (data['year'] >= 2007) & (data['year'] <= 2014),
    'val': data['year'] == 2015,
    'test': (data['year'] >= 2016) & (data['year'] <= 2017),
}

Здесь вы можете делать всевозможные преобразования признаков. 

Для того, чтобы получить качество, необходимое для преодоления бейзлайна, вам достаточно закодировать все категориальные признаки с помощью `LabelEncoder`, а также разумно обработать пропущенные значения.

In [9]:
data.isna().sum()

Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
year                 0
month                0
day                  0
dtype: int64

In [10]:
data.shape

(145460, 25)

In [11]:
data

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,year,month,day
0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,...,1007.1,8.0,NaN,16.9,21.8,No,No,2008,12,1
1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,...,1007.8,NaN,NaN,17.2,24.3,No,No,2008,12,2
2,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,...,1008.7,NaN,2.0,21.0,23.2,No,No,2008,12,3
3,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,...,1012.8,NaN,NaN,18.1,26.5,No,No,2008,12,4
4,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,...,1006.0,7.0,8.0,17.8,29.7,No,No,2008,12,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,ENE,...,1020.3,NaN,NaN,10.1,22.4,No,No,2017,6,21
145456,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,N,...,1019.1,NaN,NaN,10.9,24.5,No,No,2017,6,22
145457,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,WNW,...,1016.8,NaN,NaN,12.5,26.1,No,No,2017,6,23
145458,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,N,...,1016.5,3.0,2.0,15.1,26.0,No,No,2017,6,24


In [12]:
data['MinTemp'].fillna(value=data['MinTemp'].mean(), inplace=True)
data['MaxTemp'].fillna(value=data['MaxTemp'].mean(), inplace=True)
data['Rainfall'].fillna(value=data['Rainfall'].mean(), inplace=True)
data['Evaporation'].fillna(value=data['Evaporation'].mean(), inplace=True)
data['Sunshine'].fillna(value=data['Sunshine'].mean(), inplace=True)
data['Pressure9am'].fillna(value=data['Pressure9am'].mean(), inplace=True)
data['Pressure3pm'].fillna(value=data['Pressure3pm'].mean(), inplace=True)
data['Cloud9am'].fillna(value=data['Cloud9am'].mean(), inplace=True)
data['Cloud3pm'].fillna(value=data['Cloud3pm'].mean(), inplace=True)


In [13]:
data = data.dropna(subset=['WindGustDir'])
data = data.dropna(subset=['WindDir9am'])
data = data.dropna(subset=['WindDir3pm'])
data = data.dropna(subset=['RainTomorrow'])
data = data.dropna(subset=['RainToday'])

In [14]:
data['Humidity9am'].fillna(70, inplace=True)
data['Humidity3pm'].fillna(52, inplace=True)
data['Temp9am'].fillna(17, inplace=True)
data['Temp3pm'].fillna(20, inplace=True)

In [15]:
data.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,year,month,day
0,Albury,13.4,22.9,0.6,5.468232,7.611178,W,44.0,W,WNW,...,1007.1,8.000000,4.50993,16.9,21.8,No,No,2008,12,1
1,Albury,7.4,25.1,0.0,5.468232,7.611178,WNW,44.0,NNW,WSW,...,1007.8,4.447461,4.50993,17.2,24.3,No,No,2008,12,2
2,Albury,12.9,25.7,0.0,5.468232,7.611178,WSW,46.0,W,WSW,...,1008.7,4.447461,2.00000,21.0,23.2,No,No,2008,12,3
3,Albury,9.2,28.0,0.0,5.468232,7.611178,NE,24.0,SE,E,...,1012.8,4.447461,4.50993,18.1,26.5,No,No,2008,12,4
4,Albury,17.5,32.3,1.0,5.468232,7.611178,W,41.0,ENE,NW,...,1006.0,7.000000,8.00000,17.8,29.7,No,No,2008,12,5


In [16]:
data.isna().sum()

Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
year             0
month            0
day              0
dtype: int64

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

data.Location= le.fit_transform(data.Location.values)

In [18]:
data.WindGustDir= le.fit_transform(data.WindGustDir.values)
data.WindDir9am= le.fit_transform(data.WindDir9am.values)
data.RainToday= le.fit_transform(data.RainToday.values)
data.RainTomorrow= le.fit_transform(data.RainTomorrow.values)
data.WindDir3pm= le.fit_transform(data.WindDir3pm.values)



Ваш таргет - RainTommorow. Удалите его из обучающих данных, также удалите признак RISK_MM.

In [19]:
target_data = data['RainTomorrow']
data.drop(['RainTomorrow'], axis=1, inplace=True)

In [20]:
X_train, y_train = data[indexes['train']].values, target_data[indexes['train']].values
X_val, y_val = data[indexes['val']].values, target_data[indexes['val']].values
X_test, y_test = data[indexes['test']].values, target_data[indexes['test']].values

/home/pavel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/home/pavel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/pavel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Для каждого из алгоритмов достигнутое качество должно быть: 
* RandomForest > 0.84
* GradientBoosting > 0.845
* AdaBoost > 0.83

Обучите каждый из алгоритмов до нужного качества, используйте валидационную выборку, чтобы подбирать гиперпараметры. Получите качество (accuracy) выше необходимого и на validation, и на test.

**Подсказка:** для визуализации прогресса обучения можно использовать бибилиотеку [`tqdm`](https://tqdm.github.io/).

**Подсказка:** некоторые из подходов анасмблирования тривиальным образом поддаются распараллеливанию на несколько потоков/процессов. Для параллелизации можно использовать `multiprocessing.Pool`.

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
for i in range (2, 20):
    rf = RandomForestClassifier(n_estimators=i, max_depth=10, random_state=42)
    rf.fit(X_train, y_train)
    preds_val = rf.predict(X_val)
    preds_test = rf.predict(X_test)
    print("n_estimators = ", i)
    print("val_acc = ", accuracy_score(preds_val, y_val))
    print("test_acc = ", accuracy_score(preds_test, y_test))

n_estimators =  2
val_acc =  0.8179798771723508
test_acc =  0.799687146954028
n_estimators =  3
val_acc =  0.8405200574937933
test_acc =  0.824063613452681
n_estimators =  4
val_acc =  0.8454200966941069
test_acc =  0.8331450421482576
n_estimators =  5
val_acc =  0.850124134326408
test_acc =  0.8382289041453028
n_estimators =  6
val_acc =  0.849078792630341
test_acc =  0.8394021030676979
n_estimators =  7
val_acc =  0.8494707957663661
test_acc =  0.8388806813244112
n_estimators =  8
val_acc =  0.8528681562785836
test_acc =  0.8407491092378552
n_estimators =  9
val_acc =  0.8509081405984581
test_acc =  0.840184235682628
n_estimators =  10
val_acc =  0.8508428067424539
test_acc =  0.8399669766229252
n_estimators =  11
val_acc =  0.8489481249183327
test_acc =  0.8395324585035197
n_estimators =  12
val_acc =  0.8494707957663661
test_acc =  0.8387503258885896
n_estimators =  13
val_acc =  0.848686789494316
test_acc =  0.8391848440079951
n_estimators =  14
val_acc =  0.8494054619103619
test_

In [30]:
rf = RandomForestClassifier(n_estimators=19, max_depth=10, random_state=42)
rf.fit(X_train, y_train)
preds_val = rf.predict(X_val)
preds_test = rf.predict(X_test)
print("n_estimators = ", 19)
print("val_acc = ", accuracy_score(preds_val, y_val))
print("test_acc = ", accuracy_score(preds_test, y_test))

n_estimators =  19
val_acc =  0.854109499542663
test_acc =  0.8417050491005474


In [27]:
for i in range (2, 31):
    gbm = GradientBoostingClassifier(n_estimators=i, max_depth=5, random_state=42)
    gbm.fit(X_train, y_train)
    preds_val = gbm.predict(X_val)
    preds_test = gbm.predict(X_test)
    print("n_estimators = ", i)
    print("val_acc = ", accuracy_score(preds_val, y_val))
    print("test_acc = ", accuracy_score(preds_test, y_test))

n_estimators =  2
val_acc =  0.8188945511564093
test_acc =  0.7960806465629616
n_estimators =  3
val_acc =  0.8474454462302365
test_acc =  0.8369253497870861
n_estimators =  4
val_acc =  0.8498627989023912
test_acc =  0.8384461632050056
n_estimators =  5
val_acc =  0.8511041421664707
test_acc =  0.8399235248109846
n_estimators =  6
val_acc =  0.8520841500065334
test_acc =  0.8400104284348657
n_estimators =  7
val_acc =  0.8537828302626421
test_acc =  0.8420092117841314
n_estimators =  8
val_acc =  0.8539134979746504
test_acc =  0.8427044407751804
n_estimators =  9
val_acc =  0.8535868286946295
test_acc =  0.8418354045363692
n_estimators =  10
val_acc =  0.8546321703906965
test_acc =  0.8434431215781698
n_estimators =  11
val_acc =  0.8542401672546713
test_acc =  0.8437038324498132
n_estimators =  12
val_acc =  0.8552855089507383
test_acc =  0.8440948987572782
n_estimators =  13
val_acc =  0.8565921860708219
test_acc =  0.8445294168766837
n_estimators =  14
val_acc =  0.8561348490787927

In [28]:
gbm = GradientBoostingClassifier(n_estimators=30, max_depth=5, random_state=42)
gbm.fit(X_train, y_train)
preds_val = gbm.predict(X_val)
preds_test = gbm.predict(X_test)
print("n_estimators = ", 30)
print("val_acc = ", accuracy_score(preds_val, y_val))
print("test_acc = ", accuracy_score(preds_test, y_test))

n_estimators =  30
val_acc =  0.8596628773030184
test_acc =  0.8487007908229773
